In [1]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
data_dir = '/kaggle/input/infant-cry-audio-corpus/donateacry_corpus'
classes = ['belly_pain','burping','discomfort','hungry','tired']

In [3]:
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                data.append(mel_spectrogram)
                labels.append(i)
    
    return np.array(data), np.array(labels)


In [4]:
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [5]:
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

In [6]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.fit(X_train, y_train, epochs=30, batch_size=34, validation_data=(X_test, y_test))

Epoch 1/30
11/11 [==============================] - 7s 97ms/step - loss: 33.6116 - accuracy: 0.4849 - val_loss: 1.2082 - val_accuracy: 0.7826
Epoch 2/30
11/11 [==============================] - 0s 17ms/step - loss: 1.3283 - accuracy: 0.8466 - val_loss: 0.8939 - val_accuracy: 0.7826
Epoch 3/30
11/11 [==============================] - 0s 17ms/step - loss: 0.7235 - accuracy: 0.8493 - val_loss: 0.8238 - val_accuracy: 0.7826
Epoch 4/30
11/11 [==============================] - 0s 17ms/step - loss: 0.6213 - accuracy: 0.8493 - val_loss: 0.8473 - val_accuracy: 0.7826
Epoch 5/30
11/11 [==============================] - 0s 18ms/step - loss: 0.6115 - accuracy: 0.8493 - val_loss: 0.8234 - val_accuracy: 0.7826
Epoch 6/30
11/11 [==============================] - 0s 18ms/step - loss: 0.5865 - accuracy: 0.8493 - val_loss: 0.8611 - val_accuracy: 0.7826
Epoch 7/30
11/11 [==============================] - 0s 17ms/step - loss: 0.5796 - accuracy: 0.8521 - val_loss: 0.8770 - val_accuracy: 0.7826
Epoch 8/30
1

In [8]:
# los, acc = model.evaluate(X_test, y_test)
# print("Validation Accuracy:", acc)